In [1]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

In [2]:
survey = pd.read_csv('../data/Metro_Resident_Survey_Responses.csv')
survey.shape

FileNotFoundError: [Errno 2] No such file or directory: '../data/Metro_Resident_Survey_Responses.csv'

In [ ]:
survey.info()

#### Finding columns with less than 10 missing values - these are the ones we'll use to create the survey database

In [ ]:
cols = survey.isnull().sum()[survey.isnull().sum() < 10]

In [ ]:
cols_to_keep = list(cols.index.values)

In [ ]:
cols_to_keep

In [ ]:
survey2 = survey[cols_to_keep]

In [ ]:
survey2.head()

In [ ]:
survey2 = survey2.drop(columns = ['Start Date', 'Service District'])

In [ ]:
survey2 = survey2.rename(columns = {'End Date': 'Date'})

In [ ]:
print(survey2.Date.min())
print(survey2.Date.max())

#### Adding an `Id` column so data can be rematched with SQL joins

In [ ]:
survey2['Id'] = [i for i in range(1, 2172)]
survey2.head(3)

In [ ]:
survey2.columns.values

In [ ]:
survey_info = survey2[['Id','Period', 'Date', 'ZIP Code', 'Quality of Life',
       'Overall Feeling of Safety', 'Neighborhood Safety',
       'Neighborhood Appearance', 'Nashville as Residence',
       'Nashville as Place to Raise Children', 'Nashville as Workplace',
       'Nashville as Retirement Place', 'Gender Identity']]
survey_info.head(2)

In [ ]:
metro_gov = survey2[['Id','Metro Services - Overall',
       'Value Received from Taxes', 'Elected Official Leadership',
       'Metro Transparency and Ethics', 'Customer Service',
       'Metro Communication', 'Decision Input Opportunity']]
metro_gov.head(2)

In [ ]:
safety = survey2[['Id', 'Crime Prevention', 'Police - Overall', 'Fire and EMS - Overall', 'Police Visibility',
       'Traffic Enforcement', 'Police Professionalism',
       'Police Response Time', 'Fire Services', 'Fire Response Time',
       'Emergency Medical Services (EMS)', 'EMS Response Time',
       'Fire Professionalism', 'EMS Professionalism', '911 Services',
       'Non-Emergency Number']]
safety.head(2)

In [ ]:
general_services = survey2[['Id', 'Streets and Sidewalks - Overall', 'Trash and Recycling - Overall',
       'Public Education', 'Public Library', 'Social Services - Overall',
       'Parks and Recreation - Overall', 'Public Transit - Overall',
       'Metro-provided Arts and Culture - Overall',
       'City Street Maintenance', 'Street Signs and Traffic Signals',
       'Bicycle Infrastructure', 'Trash Service', 'Curbside Recycling',
       'Drop-off Recycling', 'Street Lighting', 'hubNashville (311)',
       'Program Information Availability', 'Nashville.gov',
       'Metro Video Programming', 'Metro Social Media']]
general_services.head(2)

In [ ]:
transportation = survey2[['Id', 'Music City Circuit', 'Music City Star', 'Park and Ride',
       'Bus Shelters']]
transportation.head(2)

In [ ]:
codes = survey2[['Id','Trash and Debris Enforcement',
       'Lawn and Brush Enforcement', 'Exterior Maintenance Enforcement',
       'Short-Term Rental Enforcement']]
codes.head(2)

In [ ]:
recreation = survey2[['Id', 'Parks Maintenance', 'Outdoor Athletic Fields',
       'Walking and Biking Trails', 'Swimming Pools', 'Community Centers',
       'Youth Recreation Programs', 'Senior Recreation Programs']]
recreation.head(2)

In [ ]:
equality_and_access = survey2[['Id', 'Healthcare Access', 'Healthy Food Access', 'Childcare Access',
       'Pre-K Access', 'Public Transit Access',
       'Diversity Support Efforts', 'Diversity Acceptance',
       'Disability Access']]

In [22]:
%who

codes	 cols	 cols_to_keep	 create_engine	 equality_and_access	 general_services	 metro_gov	 pd	 recreation	 
safety	 sqlite3	 survey	 survey2	 survey_info	 transportation	 


#### Tables to write 
 - survey_info
 - metro_gov
 - equality_and_access
 - general_services
 - safety
 - recreation
 - transportation
 - codes

In [23]:
engine = create_engine("sqlite:///metro_survey.db")

In [24]:
connection = engine.connect()
survey_info.to_sql('info', engine, if_exists = 'replace')
metro_gov.to_sql('overall', engine, if_exists = 'replace')
equality_and_access.to_sql('equality_and_access', engine, if_exists = 'replace')
general_services.to_sql('general_services', engine, if_exists = 'replace')
recreation.to_sql('recreation', engine, if_exists = 'replace')
transportation.to_sql('transportation', engine, if_exists = 'replace')
codes.to_sql('codes', engine, if_exists = 'replace')
safety.to_sql('safety', engine, if_exists = 'replace')
    
connection.close()


In [37]:
(safety.Id.value_counts() > 1).sum()

0